# Using Python to analyze financial data and test automated strategies
Computational Finance, December 11 and 13, 2017

Nicolas Mauhé

Bt is a simple Python package that can be used to fetch financial data, analyze them, establish algorithmic trading strategies, and backtesting them. It is a great tool to test anything you learn in financial courses. A complete description of the package can be found [here](http://pmorissette.github.io/bt/). It is an open-source package : which means you can access the code behind it on its [github repository](https://github.com/pmorissette/bt). Most of the Python packages work this way : so please get used to using other people work and to reading package documentations, because that is the way Python works. 

We first allow matplotlib in the notebook in order to have nice graphs, and we import bt.

In [ ]:
%matplotlib inline
import bt

Let's start by choosing which stock we want to invest in. We will focus on US equity. We can use any ticker symbol of any US Equity. A list can be found [here](http://eoddata.com/symbols.aspx). Let's choose three companies.

We can use the bt method bt.get() to fetch financial data from Yahoo. To know how a method work, we can always use the python method help().

In [ ]:
help(bt.get)

This is a bit "raw", but it can be useful. Another solution is to check the [package documentation](http://pmorissette.github.io/bt/index.html). Here is how to use bt.get():

In [ ]:
equity_list = ['AAPL', 'MCD', 'MSFT']
data = bt.get(equity_list, start='2010-01-01')
print data.head()

We can check the type of this new Python object by using the method type()

In [ ]:
type(data)

It is a dataframe of the package Pandas, the most used Python package to do data analysis.
We can check the features of a pandas dataframe by using the method dir().

In [ ]:
dir(data)

There are a lot of things, and methods are mixed up with attributes. It is easier to check the Pandas documentation about [dataframes](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html). One can see that some interesting methods are present, such as the one used to plot the data on a graph, plot().


In [ ]:
data.plot()

We can also find the method hist(), used to plot an histogram of the values.

In [ ]:
data.hist()

An interesting argument for the drawing methods such as plot() or hist() is the size you want. Just specify figsize = (length, height).

In [ ]:
data.tail(300).rebase().plot(figsize = (15, 10))

Another very useful method is describe(), that gives us summary statistics about the dataset.

In [ ]:
data.describe()

Exercise : find a way to 
1. display only the most recent prices (the last items of the dataframe). 
2. plot them....
3. Ideally, I would like to plot the evolution of these equities during their last 30 quotes.

Solution (3 commands)

Some useful bt methods are rebase() and to_returns().

In [ ]:
print data.head().rebase()

In [ ]:
print data.head().to_returns()

Exercise : produce the two following plots :
- The price evolution during the last 30 quotes rebased to 100 at the beginning of the plot.
- The histogram of the returns of the three equities during the last 30 quotes

Solution

Let's see how to display some financial statistics about our data.

In [ ]:
stats = data.calc_stats()
stats.display()

What does it mean ?

- Total Return: Total return on the period.
- Daily Sharpe: Daily Sharpe ratio

The Sharpe ratio : $$S(x) = \frac{R_x - r}{\sigma_x}$$
With $R_x$ being the investment return, $r$ being the riskfree rate and $\sigma_x$ being the standard deviation of the investment.
The Sharpe ratio measures the return you are receiving in comparison with the risk you are taking.
- Daily Sortino: Daily SOrtino ratio

The Sortino ratio (as used in the bt package) is a variation of the Sharpe ratio, taking into account only the negative volatility. The formula is very similar :
$$S^{'}(x) = \frac{R_x - r}{\sigma^-_x}$$
With $R_x$ being the investment return, $r$ being the riskfree rate and $\sigma^-_x$ being the downside deviation of the investment.

- CAGR: Compound annual growth rate

The compound annual growth rate (CAGR) :
$${CAGR}(t_0,t_n) = \left( {V(t_n)/V(t_0)} \right)^\frac{1}{t_n-t_0} - 1$$
It is an easy way to have an idea of the "average" return during the period : taking the actual average of the annual returns is not as good, given that volatility can affect the results.

- Max Drawdown: Maximum decline of the equity
- Calmar Ratio: Ratio of the CAGR and the absolute value of the Max Drawdown

 
 
- Daily Mean (ann.): Daily average of return, times the number of trading periods in a year (252)
- Daily Vol (ann.): Same thing with the standard deviation
- Daily Skew: Usual measure of the asymmetry, daily average
- Daily Kurt: Usual measure of "tailedness", daily average
 
  
  
- MTD: The month to date return
- 3m: 3 months ago to date return
- 6m: 6 months
- YTD: The beginning of the year to date return
- 1Y: One year ago to date return
- 3Y: Etc.
- 5Y: Etc.
- 10Y: Be careful : these stats are limited by the data start date !
- Since Incep. (ann.): Same thing !

To get correct statistics (such as Sharpe and Sortino ratios), we have to specify the riskfree rate on the considered period. The Treasury Bonds rate is used. We will use the ticker symbol [IEF](http://eoddata.com/stockquote/NASDAQ/IEF.htm) to determine the riskfree rate.

In [ ]:
riskfree =  bt.get('IEF', start='2010-01-01')
riskfree_rate = float(riskfree.calc_cagr())
print riskfree_rate

We can now display accurate statistics about our data.

In [ ]:
stats.set_riskfree_rate(riskfree_rate)
stats.display()

## Strategies

You can establish strategies in bt in order to choose a portfolio of equities and to automatically ajust this portfolio given a determined strategy. To do so, we will use the bt.Strategy() object.

In [ ]:
help(bt.Strategy)

As we can see, a strategy is a stack of algorithms that work one after the other, transmitting data to each other and / or stop signals. A more complete explanation can be found in the [official documentation](http://pmorissette.github.io/bt/algos.html).

A good structure for a stack of algorithm is the following one :
- frequency : which frequency should your strategy use ?
- selection: which securities should you choose ?
- weighting: how much weight should each of the selected securities have in the target portfolio?
- allocate: close out positions that are no longer needed and allocate capital to those that were selected and given target weights.

Let's take a practical example, making these choices :
- frequency : every month
- selection: all the securities available (in the dataframe)
- weighting: equal weight to all
- allocate.


In [ ]:
s1 = bt.Strategy('Equal weights', 
                       [bt.algos.RunMonthly(),
                       bt.algos.SelectAll(),
                       bt.algos.WeighEqually(),
                       bt.algos.Rebalance()])

We then run a backtest to assess the efficiency of our automated trading strategy.

In [ ]:
equal_weights = bt.Backtest(s1, data)
result = bt.run(equal_weights)
result.plot()
dir(result)

We can display the same statistics we computed before, this time for the entire portfolio (and its monthly variations).

In [ ]:
result.set_riskfree_rate(riskfree_rate)
result.display()

We can plot the weight variations to have an idea of the algorithm decisions regarding our portfolio.

In [ ]:
result.plot_security_weights()

Exercise. Let's compare this strategy to the Standard & Poor's 500 index. We can create a strategy that only buys S&P (ticker symbol : SPY)

Solution

We can now run the same backtest comparing our two strategies.

In [ ]:
# To delete
s_p_data = bt.get('SPY', start='2010-01-01')
s2 = bt.Strategy('Benchmark', 
                       [bt.algos.RunOnce(),
                       bt.algos.SelectAll(),
                       bt.algos.WeighEqually(),
                       bt.algos.Rebalance()])
# End To delete
benchmark = bt.Backtest(s2, s_p_data)
result = bt.run(equal_weights, benchmark)
result.plot()

## Selecting algos

### Frequency

The frequency can be just once using [RunOnce](http://pmorissette.github.io/bt/bt.html#bt.algos.RunOnce), a specific date using [RunOnDate](http://pmorissette.github.io/bt/bt.html#bt.algos.RunOnDate), or every n periods [RunEveryNPeriods](http://pmorissette.github.io/bt/bt.html#bt.algos.RunEveryNPeriods). The frequency can also be at the beginning of each day, week, month, quarter or year.
- [RunDaily](http://pmorissette.github.io/bt/bt.html#bt.algos.RunDaily)
- [RunWeekly](http://pmorissette.github.io/bt/bt.html#bt.algos.RunWeekly)
- Etc.

### Selection 

Let's see how we can select specific equities based on conditions. The algos we have are

- [SelectAll](http://pmorissette.github.io/bt/bt.html#bt.algos.SelectAll)
- SelectHasData
- SelectMomentum
- SelectN
- SelectRandomly
- SelectThese
- SelectWhere

We will use SelectRandomly to conduct a quick experiment. 

In [ ]:
equity_list = ['AAPL', 'MCD', 'MSFT', 'TGT', 'GE', 'AMZN', 'T', 'UPS', 'GM', 'IBM', 'PEP', 'VZ', 'DIS', 'INTC', 'FORD', 'CMCSA', 'IEF']
data = bt.get(equity_list, start='2010-01-01')
print data.head()

Let's build our strategy as before, but this time using SelectRandomly:

In [ ]:
s_10random = bt.Strategy('10 random', 
                       [bt.algos.RunMonthly(),
                       bt.algos.SelectRandomly(),
                       bt.algos.WeighEqually(),
                       bt.algos.Rebalance()])
backtest_10random = bt.Backtest(s_10random, data)
result = bt.run(backtest_10random, benchmark)
result.plot()

In [ ]:
result.set_riskfree_rate(riskfree_rate)
result.display()

Our strategy is less efficient than the S & P, we did not beat the market.

Exercise : We want to code an algorithm that follow a simple strategy : only invest in the 5 securities that have the highest return, every month, among our list of famous US securities. Write the strategy, and compare it to the random strategy and the S & P index. Did we beat the market ?

Weights:

Allocate :

- [Rebalance](http://pmorissette.github.io/bt/bt.html#bt.algos.Rebalance)
- [RebalanceOverTime](http://pmorissette.github.io/bt/bt.html#bt.algos.RebalanceOverTime)
